In [ ]:
import os, shutil

original_dataset_dir = '/home/rave/deeplearn_imagery/data/raw/stephtest/'
base_dir = '/home/rave/deeplearn_imagery/data/raw/'

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

train_fields